In [129]:
import numpy as np
from random import seed
from random import randint

In [130]:
import pandas as pd
import firebase_admin
from firebase_admin import credentials, firestore

In [131]:
import firebase_admin
from firebase_admin import credentials

cred = credentials.Certificate("famealy-a8a62-firebase-adminsdk-5xa8e-ae2b470cf7.json")
firebase_admin.initialize_app(cred)

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [3]:
db = firestore.client()

In [134]:
fname = "famealy_survey_data"
try:
    fhand = open(fname)
except:
    print('File cannot be opened:', fname)
    exit()

raw_list = list()
for line in fhand:
    raw_list.append(line.split("\t"))

In [149]:
raw_data = list()
for lists in raw_list:
    i = 1 
    while i < len(lists):
        temp_dict = dict()
        temp_dict['name'] = lists[i]
        temp_dict['foods'] = lists[i+1].split(',')
        if i == 1:
            temp_dict['type'] = 'Breakfast'
            raw_data.append(temp_dict)
        if i == 3:
            temp_dict['type'] = 'Lunch'
            raw_data.append(temp_dict)
        if i == 5: 
            temp_dict['type'] = 'Dinner'
            raw_data.append(temp_dict)
        i += 2

In [150]:
final_data = list()
for meal in raw_data:
    foods = list()
    temp = meal
    for food in meal['foods']:
        food_add = db.collection('foods').document()
        food_add.set({"name":food.lower(),"calorie":randint(5,85)})
        foods.append(food_add.id)
    temp['foods'] = foods
    final_data.append(temp)

In [151]:
for data in final_data:
    meal_add = db.collection('meals').document()
    meal_add.set({'name':data['name'],'type':data['type'],'portion':randint(1,5)})
    for food_data in data['foods']:
        food_meal_add = db.collection('meals').document(meal_add.id).collection('food').document()
        food_meal_add.set({'foodId':food_data,'quantity':randint(1,5)})

In [174]:
###################
#Delete Duplicates#
###################

docs = db.collection('foods').get()
doc_list = list()
for doc in docs:
    addable = doc.to_dict()['name'].strip()
    doc_list.append([addable,doc.id])

[['rice', '1TTIE2g8aymSIIPwvvSF'], ['lemon', '1Wzh75aUJThzZFXFUlFv'], ['bread', '1ju13ge41fiSFCsfudX2'], ['olive oil', '2otNHad27vcaMf89Fztl'], ['beef', '3ukKQJZRn2CeUAeEzQaj'], ['tomato sauce', '3vkIPXaFGmmuArf48hR7'], ['eggs', '4EoprQWAop8IpglEypcH'], ['milk', '4O7em509FxrVcsbtNA2m'], ['shredded cheese', '5Qntpf4cXcwAs5VXd58J'], ['chilli flakes', '5prMTQzX0UYuKzKrBNlm'], ['chorizo', '70y7HVAkL2vAzy38Nff5'], ['bread', '7PrmpWm9EHJyOWGp6S63'], ['mayonnaise', '8OuM1FoZ8ybWvPYTSpT6'], ['goat cheese', 'AFkBJkkIPFjVOrsLqt2P'], ['dolmio sauce', 'BfIEWZ0NAAg0KKMOWr02'], ['hash browns', 'D2MjAan9pBmGVqKV4sFd'], ['shredded cheese', 'DCb6uE71LtFvrz94F5ma'], ['carrot', 'DtIiw4jUnGZFrZesPtXg'], ['egg', 'EBnX5rovQ7MGiJeGszs7'], ['porridge oats', 'ERdrdyzD7xeYr8z3qLzT'], ['cheese', 'F15YMgMXKQMvnik5m0at'], ['onion', 'FH27cU3qQ8fiQQpRg4LK'], ['bell peppers', 'Fj8opTI5wgs3laXifpVT'], ['walnuts', 'FjN15vBrXMXr3b7A9kW9'], ['ham', 'FjYkH7tDlSZXpT6Nvrpr'], ['taco seasoning', 'H0Mz1Z2M3IGcoRAmCy17'], ['mu

In [175]:
dup_list = list()
for name in doc_list:
    i = 0
    dup_list_sub = list()
    for name_2 in doc_list:
        if name[0] == name_2[0]:
            dup_list_sub.append(name_2)
            doc_list.remove(name_2)
    if len(dup_list_sub) > 1:
        dup_list.append(dup_list_sub)

[[['rice', '1TTIE2g8aymSIIPwvvSF'], ['rice', 'f5rXFohI8jTM80OeX5IS']], [['bread', '1ju13ge41fiSFCsfudX2'], ['bread', '7PrmpWm9EHJyOWGp6S63'], ['bread', 'JFk8rj7obCo5Is5i8F9m'], ['bread', 'VTZNxDXjOTEygrfcyCWo'], ['bread', 'Z6UImUt1bVpAtFrLuGDp'], ['bread', 'xKmXYh21HitJqwSDSrCx']], [['eggs', '4EoprQWAop8IpglEypcH'], ['eggs', 'SCAOspOG3AGt2sKktaGw'], ['eggs', 'WMy0zf6SUwfShjKe7DX4'], ['eggs', 'vfMmSjCaDwD1i2N2YkAe']], [['shredded cheese', '5Qntpf4cXcwAs5VXd58J'], ['shredded cheese', 'DCb6uE71LtFvrz94F5ma']], [['cheese', 'F15YMgMXKQMvnik5m0at'], ['cheese', 'Jyb6y8RMIuyUZ6MIenMs'], ['cheese', 'QB08iLqLdsgMObIlCNRt'], ['cheese', 'TBqwQCTF4jWjPuD10O8f'], ['cheese', 'yueRPvkxKBtsNHMLG8ks']], [['ham', 'FjYkH7tDlSZXpT6Nvrpr'], ['ham', 'bnK0hqggZbkUtS2XwoI1']], [['chopped tomatoes', 'MVN13NafvPvtqkGFghh3'], ['chopped tomatoes', 'XVyVQ3k0XQvkrM1t0JHK']], [['onion', 'FH27cU3qQ8fiQQpRg4LK'], ['onion', 'O6wvuFe8mqWK0COWOjwO'], ['onion', 'Sl0QiIo5nf4sA56nPMy1'], ['onion', 'gF1XCaGnBqMaRmLBwbTp']], [

In [165]:
for lists in dup_list:
    i = 1
    while i < len(lists):
        dp.collection('foods').document(lists[i][1]).delete()
        meal_tag = dp.collection('meals').get()
        for meal in meal_tag:
            dp.collection('meals').document(meal.id).collection('food').where('foodId','==',lists[i][1]).update('foodId':lists[0][1])
        

['b', 'c', 'a']
